In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_io import load_excel_data, clean_observations, get_annual_summary

# set some plotting defaults
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## Survey Protocol

This dataset follows a standardized breeding bird monitoring protocol:

- **Transect structure**: Each site contains **10 fixed observation points** arranged along a transect
- **Visit protocol**: Observers conduct **5-minute point counts** at each of the 10 points
- **Visitation frequency**: Each site is visited **at most twice** during the monitoring period
- **Detection recording**: All bird detections include species ID, count, and observer metadata

This standardized approach ensures comparable data across sites and years, though sampling effort varies (see below).

In [ ]:
# load the data
data = load_excel_data('../data/raw/Observations 2012-2025.xlsx')
df_obs = data['observations']
df_species = data['species']
df_gps = data['gps']

In [ ]:
# take a look at the structure
print("Observations shape:", df_obs.shape)
print("\nColumn names:")
print(df_obs.columns.tolist())
print("\nFirst few rows:")
df_obs.head()

In [ ]:
# clean it up
df_clean = clean_observations(df_obs)

In [ ]:
# Data Quality Issues and Solutions
print("Critical Data Quality Issues")

print("1. DATETIME ENCODING OF COUNTS")
print("   Problem: Count data was stored as datetime64 (Excel import artifact)")
print("   Solution: Extracted counts from nanosecond component of timestamps")
print("   Impact: Affected 100% of observations - now corrected\n")

print("2. PARTIAL YEARS")
print(f"   2014: {df_clean[df_clean.year == 2014]['date'].min().strftime('%B %d')} to {df_clean[df_clean.year == 2014]['date'].max().strftime('%B %d')} (partial)")
print(f"   2025: {df_clean[df_clean.year == 2025]['date'].min().strftime('%B %d')} to {df_clean[df_clean.year == 2025]['date'].max().strftime('%B %d')} (partial)")
print("   Recommendation: Focus analyses on complete years 2015-2024\n")

print("3. OBSERVER EFFECT")
top_observer = df_clean.groupby('observer_name').size().sort_values(ascending=False).iloc[0]
top_obs_name = df_clean.groupby('observer_name').size().sort_values(ascending=False).index[0]
obs_pct = (top_observer / len(df_clean)) * 100
print(f"   One observer ({top_obs_name}) conducted {obs_pct:.1f}% of all observations")
print("   Impact: Potential observer-specific detection biases\n")

print("4. MISSING SPECIES METADATA")
n_species_obs = df_clean['species_name'].nunique()
n_species_lookup = len(df_species)
print(f"   {n_species_obs} species observed, but only {n_species_lookup} in lookup table")
print(f"   {n_species_obs - n_species_lookup} species lack scientific names and migration status\n")

print("5. DISTANCE BANDS")
print("   Distance sampling data is 70-99% missing")
print("   Analysis will focus on total counts only\n")

print("6. NEGATIVE WIND VALUES")
print(f"   Found {(df_obs['wind'] < 0).sum()} observations with negative wind values (data errors)")
print("   Solution: Set to NaN while retaining bird observations")


In [ ]:
# get annual summary
annual_summary = get_annual_summary(df_clean)
annual_summary

In [ ]:
# Basic Dataset Overview
print("Basic Dataset Overview")

print("\nTemporal Coverage:")
print(f"  First observation: {df_clean['date'].min().strftime('%Y-%m-%d')}")
print(f"  Last observation:  {df_clean['date'].max().strftime('%Y-%m-%d')}")
print(f"  Total years:       {df_clean['year'].nunique()}")
print(f"  Year range:        {int(df_clean['year'].min())} - {int(df_clean['year'].max())}")

print("\nSpatial Coverage:")
print(f"  Unique transects:  {df_clean['transect_name'].nunique()}")
print(f"  Unique points:     {df_clean['point_number'].nunique()} (1-10 per transect)")

print("\nSpecies Diversity:")
print(f"  Unique species:    {df_clean['species_name'].nunique()}")
print(f"  Total individuals: {df_clean['individual_count'].sum():,.0f}")

print("\nSampling Effort:")
print(f"  Total observations: {len(df_clean):,}")
print(f"  Unique observers:   {df_clean['observer_name'].nunique()}")

print("\nCount Statistics:")
print(f"  Mean birds/observation:   {df_clean['individual_count'].mean():.1f}")
print(f"  Median birds/observation: {df_clean['individual_count'].median():.0f}")
print(f"  Max birds/observation:    {df_clean['individual_count'].max():.0f}")

In [ ]:
# Missing Value Analysis (Essential Columns Only)
print("Missing Value Analysis\n")

essential_cols = ['observer_name', 'transect_name', 'species_name', 'date', 
                  'year', 'individual_count', 'point_number', 'visit_number']

missing_summary = df_clean[essential_cols].isnull().sum()
missing_pct = (missing_summary / len(df_clean) * 100).round(2)

print("Essential columns:")
for col in essential_cols:
    if missing_summary[col] == 0:
        print(f"  {col:20s}: ✓ Complete")
    else:
        print(f"  {col:20s}: {missing_summary[col]:,} missing ({missing_pct[col]}%)")

print(f"\nWeather columns (wind, rain, visibility): <1% missing - acceptable")
print(f"Distance bands: 70-99% missing - will not use in analysis")
print(f"\n✓ Dataset quality is suitable for biodiversity analysis")

In [ ]:
# Simple Temporal Coverage Plot
fig, ax = plt.subplots(figsize=(12, 6))

years = annual_counts.index.values
obs_counts = annual_counts['N_Obs'].values

# Colour partial years differently
colors = ['orange' if year in [2014, 2025] else 'blue' for year in years]

ax.bar(years, obs_counts, color=colors, edgecolor='black', alpha=0.8)
ax.set_xlabel('Year', fontweight='bold', fontsize=12)
ax.set_ylabel('Number of Observations', fontweight='bold', fontsize=12)
ax.set_title('Observation Effort Over Time', fontweight='bold', fontsize=14)
ax.grid(axis='y', alpha=0.3)

# Mark partial years
for year in [2014, 2025]:
    if year in years:
        idx = list(years).index(year)
        ax.text(year, obs_counts[idx] + 200, 'Partial\nYear', 
                ha='center', fontsize=9, color='darkred', fontweight='bold')

plt.tight_layout()
plt.savefig('../figures/eda_temporal_coverage.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: ../figures/eda_temporal_coverage.png")

In [ ]:
# Spatial Coverage Plot
transects_per_year = df_clean.groupby('year')['transect_name'].nunique()
total_transects = df_clean['transect_name'].nunique()

fig, ax = plt.subplots(figsize=(12, 6))
coverage_pct = (transects_per_year / total_transects * 100)

ax.bar(transects_per_year.index.astype(int), coverage_pct.values, 
       color='teal', edgecolor='black', alpha=0.8)
ax.axhline(y=100, color='red', linestyle='--', label='Full coverage')
ax.set_xlabel('Year', fontweight='bold', fontsize=12)
ax.set_ylabel('Spatial Coverage (%)', fontweight='bold', fontsize=12)
ax.set_title(f'Spatial Coverage by Year ({total_transects} transects total)', 
             fontweight='bold', fontsize=14)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../figures/eda_spatial_coverage.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: ../figures/eda_spatial_coverage.png")

### Species: simple summaries and plots

Below we summarise species-level totals and richness over time to understand which species dominate counts and how diversity changes annually.

In [ ]:
# Top 20 species by total individuals
species_totals = (
    df_clean.groupby('species_name')['individual_count']
            .sum()
            .sort_values(ascending=False)
)

top_n = 20
fig, ax = plt.subplots(figsize=(10, 8))
ax.barh(species_totals.head(top_n).index[::-1],
        species_totals.head(top_n).values[::-1],
        color=sns.color_palette(None, top_n))
ax.set_xlabel('Total individuals (2014-2025)', fontweight='bold')
ax.set_ylabel('Species', fontweight='bold')
ax.set_title(f'Top {top_n} species by abundance', fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/eda_species_top20.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: ../figures/eda_species_top20.png')


In [ ]:
# Annual species richness (unique species per year)
richness_by_year = (
    df_clean.groupby('year')['species_name']
            .nunique()
            .sort_index()
)

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(richness_by_year.index.astype(int), richness_by_year.values,
        marker='o', linewidth=2, color='tab:green')
ax.set_xlabel('Year', fontweight='bold')
ax.set_ylabel('Unique species', fontweight='bold')
ax.set_title('Species richness by year', fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# highlight partial years if present
for year in [2014, 2025]:
    if year in richness_by_year.index:
        ax.scatter([year], [richness_by_year.loc[year]], color='orange', s=60, zorder=3)
        ax.text(year, richness_by_year.loc[year] + 1, 'Partial', ha='center', fontsize=9, color='darkred')

plt.tight_layout()
plt.savefig('../figures/eda_species_richness_by_year.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: ../figures/eda_species_richness_by_year.png')


### Observer effort: simple summaries and plots

We assess how sampling effort is distributed across observers and over time.


In [ ]:
# Observer Effort Analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Observations
obs_by_observer = df_clean.groupby('observer_name').size().sort_values(ascending=False)
n = 10
ax1.barh(obs_by_observer.head(n).index[::-1], obs_by_observer.head(n).values[::-1])
ax1.set_xlabel('Number of observations', fontweight='bold')
ax1.set_title(f'Top {n} Observers by Observation Count', fontweight='bold')

# Transects
transects_by_observer = df_clean.groupby('observer_name')['transect_name'].nunique().sort_values(ascending=False)
ax2.barh(transects_by_observer.head(n).index[::-1], transects_by_observer.head(n).values[::-1], color='teal')
ax2.set_xlabel('Unique transects', fontweight='bold')
ax2.set_title(f'Top {n} Observers by Spatial Coverage', fontweight='bold')

plt.tight_layout()
plt.savefig('../figures/eda_observer_effort.png', dpi=300, bbox_inches='tight')
plt.show()